In [24]:
import pandas as pd
import numpy as np
import pickle

In [25]:
test = pd.read_csv("validation_for students.csv")

In [26]:
test.head()

,index,store_ID,day_of_week,date,nb_customers_on_day,open,promotion,state_holiday,school_holiday
0,272371,415,7,01/03/2015,0,0,0,0,0
1,558468,27,7,29/12/2013,0,0,0,0,0
2,76950,404,3,19/03/2014,657,1,1,0,0
3,77556,683,2,29/01/2013,862,1,0,0,0
4,456344,920,3,19/03/2014,591,1,1,0,0


In [27]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71205 entries, 0 to 71204
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   index                71205 non-null  int64 
 1   store_ID             71205 non-null  int64 
 2   day_of_week          71205 non-null  int64 
 3   date                 71205 non-null  object
 4   nb_customers_on_day  71205 non-null  int64 
 5   open                 71205 non-null  int64 
 6   promotion            71205 non-null  int64 
 7   state_holiday        71205 non-null  object
 8   school_holiday       71205 non-null  int64 
dtypes: int64(7), object(2)
memory usage: 4.9+ MB


In [28]:
xgb_model_simple = pickle.load(open('simple_model.pkl','rb'))


In [29]:


x_test = test[['nb_customers_on_day', 'open', 'promotion']]


y_pred = xgb_model_simple.predict(x_test)


y_pred

array([4.8743241e-02, 4.8743241e-02, 6.8970400e+03, ..., 5.8300635e+03,
       6.6497798e+03, 8.3169004e+03], dtype=float32)

In [30]:
predictions_df = pd.DataFrame({'Index': test['index'], 'Prediction': y_pred})

predictions_df.to_csv('simple_predictions_new_data.csv', index=False)

In [31]:
test['date'] = pd.to_datetime(test['date'], format= '%d/%m/%Y')
test['year'] = pd.to_datetime(test['date']).dt.year
test['month'] = pd.to_datetime(test['date']).dt.month

In [32]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse=False, drop='first') 

# Fit and transform the categorical columns
encoded_data = encoder.fit_transform(test[["day_of_week", "state_holiday", "year", "month"]])

encoded_data

# Create a DataFrame with the encoded data and column names
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(["day_of_week", "state_holiday", "year", "month"]))

# Concatenate the encoded DataFrame with the original DataFrame
data_encoded = pd.concat([test, encoded_df], axis=1)

# Drop the original categorical columns if needed
data_encoded = data_encoded.drop(columns =["day_of_week", "state_holiday", "year", "month"], axis=1)

c:\Users\ruoxi\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [33]:
data_encoded = data_encoded.drop(columns =['date'], axis=1)

In [34]:
xgb_model = pickle.load(open('model.pkl','rb'))

In [35]:
data_encoded.columns

Index(['index', 'store_ID', 'nb_customers_on_day', 'open', 'promotion',
       'school_holiday', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4',
       'day_of_week_5', 'day_of_week_6', 'day_of_week_7', 'state_holiday_a',
       'state_holiday_b', 'state_holiday_c', 'year_2014', 'year_2015',
       'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7',
       'month_8', 'month_9', 'month_10', 'month_11', 'month_12'],
      dtype='object')

In [37]:

x_test = data_encoded[['store_ID', 'nb_customers_on_day', 'open', 'promotion',
       'school_holiday', 'day_of_week_2', 'day_of_week_3', 'day_of_week_4',
       'day_of_week_5', 'day_of_week_6', 'day_of_week_7', 'state_holiday_a',
       'state_holiday_b', 'state_holiday_c', 'year_2014', 'year_2015',
       'month_2', 'month_3', 'month_4', 'month_5', 'month_6', 'month_7',
       'month_8', 'month_9', 'month_10', 'month_11', 'month_12']]


y_pred = xgb_model.predict(x_test)

predictions_df = pd.DataFrame({'Index': data_encoded['index'], 'Prediction': y_pred})

predictions_df.to_csv('complex_predictions_new_data.csv', index=False)

In [39]:
pip freeze > requirements_00.txt

Note: you may need to restart the kernel to use updated packages.
